In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']
SERPAPI_API_KEY = config['SERPAPI']['SERPAPI_API_KEY']


os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})
os.environ.update({'SERPAPI_API_KEY': SERPAPI_API_KEY})

### Get tools

In [3]:
from models.llm.chain import GraphChain, DraftChunkChain

In [4]:
graph_template_prompt_path = '../../openai_skt/models/templates/graph_prompt_template.txt'
with open(graph_template_prompt_path, 'r') as f:
    graph_template = f.read()
graph_chain = GraphChain(graph_template=graph_template, input_variables=["graph_to_draw"])

In [5]:
draft_chunk_template_prompt_path = '../../openai_skt/models/templates/draft_chunk_prompt_template.txt'
with open(draft_chunk_template_prompt_path, 'r') as f:
    draft_chunk_template = f.read()
draft_chunk_chain = DraftChunkChain(draft_chunk_template=draft_chunk_template, input_variables=["draft", "query"])

In [22]:
from tools import DatabaseTool, SearchTool, SearchByURLTool, TimeTool, DraftChunkTool, GraphTool
from models.llm.chain import SummaryChunkChain

In [17]:
summary_chunk_template = '''
I want you to answer the question from the given documentation. 
There may be a lot of information in a given document that is not relevant to your question, but you should try to find as much relevant information as possible.
Be as diligent as possible in returning your answer based on your knowledge and documentation.
If there is no answer in the document, return "No useful information".

Document: {document}
Question: {question}
Answer:
'''

In [35]:
summary_chunk_chain = SummaryChunkChain(summary_chunk_template=summary_chunk_template, input_variables=["question", "document"], verbose=True)
search_by_url_tool = SearchByURLTool()

In [24]:
from langchain.agents import Tool
from langchain.utilities import SerpAPIWrapper

# You can create the tool to pass to an agent
search = SerpAPIWrapper(params={
        "engine": "google",
        "google_domain": "google.com",
        "gl": "kr",
        "hl": "ko",
        })
search_tool = Tool(
    name="search_tool",
    description="A tool to search on internet.",
    func=search.run,
)

In [36]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
draft_chunk_tool = DraftChunkTool(draft_chunk_chain=draft_chunk_chain)
graph_tool = GraphTool(graph_chain=graph_chain)
search_tool = SearchTool(summary_chunk_chain=summary_chunk_chain, search_by_url_tool=search_by_url_tool)
time_tool = TimeTool()

In [37]:
tools = [database_tool, graph_tool, search_tool]

### Set Agent

In [9]:
import re
from typing import List, Union

from langchain import LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.tools import BaseTool

In [10]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[BaseTool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
class DraftEditAgent:
    # TODO: Database 객체로 선언 안하는 방법이 있는지 생각
    def __init__(self, tools, draft_edit_prompt_path='../openai_skt/models/templates/draft_edit_prompt_template.txt', verbose=False) -> None:
        with open(draft_edit_prompt_path, 'r') as f:
            self.draft_edit_prompt_template = f.read()
        
        self.output_parser = CustomOutputParser()
        self.verbose = verbose
        self.tools = tools

        self.draft_edit_prompt = CustomPromptTemplate(
            template=self.draft_edit_prompt_template,
            tools=self.tools,
            # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
            # This includes the `intermediate_steps` variable because that is needed
            input_variables=["user_query", "draft", "intermediate_steps"]
        )

        self.llm = ChatOpenAI(model='gpt-3.5-turbo-16k', temperature=0, verbose=self.verbose)
        self.draft_edit_chain = LLMChain(llm=self.llm, prompt=self.draft_edit_prompt, verbose=self.verbose)
        tool_names = [tool.name for tool in self.tools]
        self.agent = LLMSingleActionAgent(
            llm_chain=self.draft_edit_chain, 
            output_parser=self.output_parser,
            stop=["\nObservation:"], 
            allowed_tools=tool_names
        )
        self.agent_executor = AgentExecutor.from_agent_and_tools(agent=self.agent, tools=self.tools, verbose=self.verbose)

    def run(self, database, draft, query):
        part_draft = draft_chunk_chain.run(draft=draft, query=query)
        input_dict = self.parse_input(database, part_draft, query)
        result = self.agent_executor.run(input_dict)
        return result

    async def arun(self, database, draft, query):
        input_dict = self.parse_input(database, draft, query)
        result = await self.agent_executor.arun(input_dict)
        return result
    
    def parse_input(self, database, draft, query):
        database_tool = self.tools[0]
        database_tool.set_database(database)
        input_dict = {'user_query': query, 'draft': draft}
        return input_dict

In [38]:
draft_edit_agent = DraftEditAgent(tools=tools, verbose=True, draft_edit_prompt_path='../../openai_skt/models/templates/draft_edit_prompt_template.txt')

In [12]:
from database import DataBase

In [13]:
from database import CustomEmbedChain
embed_chain = CustomEmbedChain()

In [ ]:
database = DataBase.load(database_path='./user/test_2/database.json', embed_chain=embed_chain)

In [15]:
with open('./user/test_2/draft_0.md', 'r') as f:
    draft = f.read()

In [39]:
draft_edit_agent.run(database=database, draft=draft, query="결론 부분에 2023년 비트코인 가격 차트를 넣어줘")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You should modify the draft according to the user's requirements. Use a variety of tools to answer the questions.
You need to find the material first, then use it to answer.

You have access to the following tools:

database: A tool to get data with from database. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must contain both a query and a question.
graph_tool: A tool to draw a graph. This tool simply converts the data you give it into a graph, so you'll need to enter a table in markdown form with your data and the characteristics of the graph you want to draw. It return image path of the graph.
search_tool: A tool to search 

'비트코인은 미래에 더욱 중요한 역할을 할 것으로 예상되며, 암호화폐 시장의 발전과 함께 더욱 중요한 자산으로 인정받을 것입니다. 디지털 자산 시장의 발전과 함께 비트코인은 더욱 더 중요한 역할을 할 것으로 예상됩니다. 따라서 비트코인에 대한 이해와 관련된 기술과 시스템의 발전이 필요하며, 국내외의 금융 기관과의 협력이 필요합니다. 이를 통해 우리나라는 비트코인이 촉발한 글로벌 시장의 변화에 뒤처지지 않고 먼저 대응할 수 있을 것입니다.'